In [ ]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [23]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

Enter a class to grade (i.e. class1 for class1)
Successfully opened class1.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [24]:
#Make a copy dataframe to check how many invalid lines of data have been removed
new_grades_df = grades.copy()
#Loop over the 'grades' data, check if length of id is not 9 and the elements of each iteration is not 26. If yes, print then remove.
for grade in new_grades_df: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                corrupted_ID = value[0]
                print("Invalid line of data:", corrupted_ID, " does not contain exactly 26 values:")
                print(grade)
                new_grades_df.remove(grade)
            id = value[0]
            id_length = len(id)
            if id_length != 9:
                corrupted_ID = value[0]
                print("Invalid line of data:", corrupted_ID, " is invalid")
                print(grade)
                new_grades_df.remove(grade)
            id_chars= list(id)
            id_chars.pop(0)
            for char in id_chars:
                    if char.isnumeric() ==0:
                        print("Invalid line of data:", id, " is invalid")
                        print(grade)
                        new_grades_df.remove(grade)

                
values_false = len(grades) - len(new_grades_df)
num_students = len(new_grades_df)
print("Total valid lines of data:", num_students)
print("Total invalid lines of data:", values_false)

#https://pandas.pydata.org/docs/reference/api/pandas.Series.str.isnumeric.html


Total valid lines of data: 20
Total invalid lines of data: 0


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------
Student of highscore 
Print result out
!!!!

In [28]:
answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_keys.split(",")
answer_key_table = pd.DataFrame(answer_key)
answers_checked = []
score_list = []
highScore_loop = []
for grade in new_grades_df:
        value = grade.split(",")
        id = value[0]
        #Create "answer_table" data frame from the grades dataframe and remove student id from data frame
        answer_table = pd.DataFrame(value)
        answer_table = answer_table.iloc[1: , :]
        answer_table = answer_table.reset_index(drop=True)
        #Remove \n from value
        cVal = answer_table.iloc[24][0]
        cValSplit = cVal.split("\n")
        c = cValSplit[0]
        answer_table = answer_table.replace([cVal],c)
        #Compare student answers with answer keys to a new column called "checkAnswer", return True if correct, return the correct answer if false and "" if blank
        conditions  = [answer_table[0] == answer_key_table[0], answer_table[0] == "" , answer_table[0] != answer_key_table[0]]
        choices     = [ "True", '',answer_key_table[0]]
        answer_table["checkAnswer"] = np.select(conditions, choices, default=np.nan)
        #Calculate the quantity of correct, false and blank answers.
        num_correct = (answer_table['checkAnswer']=='True').sum()
        num_blank = (answer_table['checkAnswer']=='').sum()
        num_incorrect = len(answer_table[answer_table['checkAnswer'].map(lambda x: x !='True' and x !='')])
        #Calculate the score
        score = num_correct*4 + num_incorrect*(-1)
        #Make a new dataframe with 2 columns, id and score, then export the dataframe outside loop
        score_pd = pd.DataFrame({'id': [id], 'score': [score]})
        score_list.append(score_pd)
        answers_checked.append(answer_table)
score_pd = pd.concat(score_list)
# Get the number of students with high score(>80)
high_score_pd = score_pd[score_pd['score'] > 80]
high_score_count = len(high_score_pd)
print("Total student of high scores:",high_score_count)
# Calculate max,min, mean, range of score, and median
mean = round(score_pd['score'].mean(),2)
max = score_pd['score'].max()
min = score_pd['score'].min()
RoS = max - min
score_pd_sorted = score_pd.sort_values(by=['score'])
score_pd_sorted = score_pd_sorted.reset_index(drop=True)
med_row = score_pd_sorted.median()
med= med_row[0]
print("Mean (average) score:", mean)
print("Highest score:", max)
print("Lowest score:", min)
print("Range of scores:", RoS)
print("Median score:", med)
answer_pd = pd.concat(answers_checked)
answer_pd.columns = ['studentAnswer', 'checkAnswer']
answer_pd['question'] = answer_pd.index


#Find out the question students most answer incorrectly and how many time the question is incorrect.
answers_incorrect_pd = answer_pd[answer_pd['checkAnswer'] != "True"]
answers_incorrect_pd = answers_incorrect_pd[answers_incorrect_pd['studentAnswer'] != ""]
answer_incorrect_count = answers_incorrect_pd.groupby('question')['question'].count()
question_most_incorrect = answer_incorrect_count.idxmax()
num_answer_incorrect = answer_incorrect_count.max()
ratio_incorrect = round(num_answer_incorrect / num_students,2)
print("Question that most people answer incorrectly:", question_most_incorrect)
print("Question", question_most_incorrect, "is answered incorrectly", num_answer_incorrect, "times","-",ratio_incorrect)
print("----------------------")

#Find out the question students most leave blank and how many time the question is leave blank.
answers_blank_pd = answer_pd[answer_pd['studentAnswer'] == ""]
answers_blank_count = answers_blank_pd.groupby('question')['question'].count()
question_most_leave_blank = answers_blank_count.idxmax()
num_answer_blank = answers_blank_count.max()
ratio_blank = round(num_answer_blank/ num_students,2)
print("Question that most people skip:", question_most_leave_blank)
print("Question", question_most_leave_blank, "is skipped", num_answer_blank, "times","-",ratio_blank)






Total student of high scores: 6
Mean (average) score: 75.6
Highest score: 91
Lowest score: 59
Range of scores: 32
Median score: 73.0
Question that most people answer incorrectly: 9
Question 9 is answered incorrectly 4 times - 0.2
----------------------
Question that most people skip: 2
Question 2 is skipped 4 times - 0.2


C:\Users\thoma\AppData\Local\Temp\ipykernel_5128\3236540744.py:45: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  med_row = score_pd_sorted.median()


Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'

In [30]:
#  with open(name+"_grades'.txt','w') as gs:
#         grades = gs.readlines()
#         print("Successfully opened", name+'.txt')
# score_df = score_df.reset_index()
for i in range(len(score_pd)):
    print(score_pd.loc[i, "id"], score_pd.loc[i, "score"])


0    N00000001
0    N00000002
0    N00000003
0    N00000004
0    N00000005
0    N00000006
0    N00000007
0    N00000008
0    N00000009
0    N00000010
0    N00000011
0    N00000012
0    N00000013
0    N00000014
0    N00000015
0    N00000016
0    N00000017
0    N00000018
0    N00000019
0    N00000020
Name: id, dtype: object 0    59
0    70
0    84
0    73
0    83
0    66
0    88
0    67
0    86
0    73
0    86
0    73
0    73
0    78
0    72
0    91
0    66
0    78
0    78
0    68
Name: score, dtype: int64


KeyError: 1